### Meteorologia: Tiempo durante el 2021

In [1]:
# Librerias para usar
import pandas as pd
import numpy as np
import datetime as dt

In [38]:
# Importando los CSV
df = pd.read_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\Datos.csv', sep = ';')
Pruebas_calidad = pd.read_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\Pruebas_calidad.csv', sep = ';')
Punto_cardinal = pd.read_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\Punto_cardinal.csv', sep = ';')
Estacion_astronomica = pd.read_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\Estacion_astronomica.csv', sep = ';')
# Cambio de tipo de dato
df.Fecha = pd.to_datetime(df.Fecha, format="%d/%m/%Y %H:%M")
df['Dia'] = pd.to_datetime(df.Fecha).dt.date
df['Hora'] = pd.to_datetime(df.Fecha).dt.time
Estacion_astronomica.Fecha = pd.to_datetime(Estacion_astronomica.Fecha, format="%d/%m/%Y %H:%M")

Datos diarios

In [55]:
# Inicio de la consulta
Consulta1 = pd.DataFrame()
# Temperatura diaria promedio, maxima y minima
Consulta1['Tdp'] = df.set_index('Fecha').groupby(pd.Grouper(freq='d')).Th.mean()
Consulta1['Tdmax'] = df.groupby('Dia').Th.max()
Consulta1['Tdmin'] = df.groupby('Dia').Th.min()
# Precipitacion acumulada diaria
Consulta1['Pd'] = df.groupby('Dia').Ph.sum() 
# Humedad relativa promedio diaria
Consulta1['Hdp'] = df.groupby('Dia').Hh.mean()

Consulta1 = Consulta1.reset_index()

Pruebas 

In [58]:
# Preparando datos
#Pruebas horarias

#Prueba tipo LD
def LD(col,min,max,col2):
 df[col2] = np.where(np.isnan(df[col]),np.nan,np.where((df[col]>= min) & (df[col]<= max),'C','M'))    

#Prueba tipo LN
def LN(col,min,max,col2):
 df[col2] = np.where(np.isnan(df[col]),np.nan,np.where((df[col]>= min) & (df[col]<= max),'C','D'))

#Prueba tipo PA
def PA(col,tiempo,rango,col2):
 a = np.where(df.Fecha.shift(tiempo) + dt.timedelta(hours=tiempo) == df.Fecha,abs(df[col].shift(tiempo) - df[col]),np.nan)
 df[col2] = np.where (np.isnan(a),np.nan,np.where(a <= rango,'C','D'))

#Prueba tipo PER1
def PER1(col,col2): 
 m1 = np.where(df.Fecha.shift(1) + dt.timedelta(hours=1) == df.Fecha,df[col].shift(1),np.nan)
 m2 = np.where(df.Fecha.shift(2) + dt.timedelta(hours=2) == df.Fecha,df[col].shift(2),np.nan)
 m3 = np.where(df.Fecha.shift(3) + dt.timedelta(hours=3) == df.Fecha,df[col].shift(3),np.nan)
 o = np.where(np.isnan(df[col]),np.nan, df[col] == df[col])
 a = np.where(np.isnan(m1),np.nan, df[col] == m1)
 b = np.where(np.isnan(m2),np.nan, df[col] == m2)
 c = np.where(np.isnan(m3),np.nan, df[col] == m3)
 df[col2] = np.where((np.isnan(m3) & np.isnan(m2) & np.isnan(m1))|(np.isnan(m1) & np.isnan(m2))|(np.isnan(m3) & np.isnan(m2)),np.nan ,np.where(o+a+b+c == 4,'D','C'))

#Prueba tipo PER2
def PER2(col,col2):
 m1 = np.where(df.Fecha.shift(1) + dt.timedelta(hours=1) == df['Fecha'],df[col].shift(1),np.nan)
 m2 = np.where(df.Fecha.shift(2) + dt.timedelta(hours=2) == df['Fecha'],df[col].shift(2),np.nan)
 m3 = np.where(df.Fecha.shift(3) + dt.timedelta(hours=3) == df['Fecha'],df[col].shift(3),np.nan)
 m4 = np.where(df.Fecha.shift(4) + dt.timedelta(hours=4) == df['Fecha'],df[col].shift(4),np.nan)
 m5 = np.where(df.Fecha.shift(5) + dt.timedelta(hours=5) == df['Fecha'],df[col].shift(5),np.nan)
 p = (m1+m2+m3+m4+m5)/5
 desv = np.sqrt((((m1-p)**2) + ((m2-p)**2) + ((m3-p)**2) + ((m4-p)**2) + ((m5-p)**2))/4)
 df[col2] = np.where((np.isnan(df[col]) | np.isnan(m1) | np.isnan(m2) | np.isnan(m3) | np.isnan(m4) | np.isnan(m5)),np.nan,(np.where(((p-3*desv) <= df[col]) & (df[col] <= (p+3*desv)),'C','D')))     

# Filtro de pruebas
def TC(a,b,col2):
 for x in range(len(df)):
  df.loc[x,col2] = np.where(any(df.iloc[x,a:b] == 'nan'),np.nan,np.where(any(df.iloc[x,a:b] == 'M'),'M',np.where(any(df.iloc[x,a:b] == 'D'),'D','C')))

# Pruebas con funciones Th(8:17) HRh(17:22) PTh(22:24)
LD('Th',-40,60,'LD_Th'), LN('Th',-32,45,'LN_Th'), PA('Th',1,4,'PA_Th1'), PA('Th',2,7,'PA_Th2'), PA('Th',3,9,'PA_Th3'), PA('Th',6,15,'PA_Th6'), PA('Th',12,25,'PA_Th12'), PER1('Th','PER1_Th'), PER2('Th','PER2_Th')
LD('Hh',0,100,'LD_Hh'), LN('Hh',5,100,'LN_Hh'), PA('Hh',1,45,'PA_Hh'), PER1('Hh','PER1_Hh'), PER2('Hh','PER2_Hh')   
LD('Ph',0,401,'LD_Ph'), LN('Ph',0,85,'LN_Ph')
TC(8,17,'TC_Th'), TC(17,22,'TC_Hh'), TC(22,24,'TC_Ph')

#Pruebas diarias
Consulta1['LD_Pd'] = np.where(np.isnan(Consulta1.Pd),np.nan,np.where((Consulta1.Pd>= 0) & (Consulta1.Pd<= 508),'C','M'))
Consulta1['LN_Pd'] = np.where(np.isnan(Consulta1.Pd),np.nan,np.where((Consulta1.Pd>= 0) & (Consulta1.Pd<= 372),'C','M'))
for x in range(len(Consulta1)):
 Consulta1.loc[x,'TC_Pd'] = np.where(any(Consulta1.iloc[x,6:8] == 'nan'),np.nan,np.where(any(Consulta1.iloc[x,6:8] == 'M'),'M',np.where(any(Consulta1.iloc[x,6:8] == 'D'),'D','C')))    

Direccion y Velocidad del Viento

In [11]:
# Preparando datos
V = df.Direccion_viento
# Reemplazo por punto cardinal
a = list(Punto_cardinal.Direccion_viento)
b = list(Punto_cardinal.Direccion_viento)[1:]
b.append(360.5)
c = list(Punto_cardinal.Punto_cardinal)
d = list(Punto_cardinal.index)
for a,b,c in zip(a,b,c):   
  V = V.replace(to_replace = np.arange(a,b,0.5), value = c)
c = list(Punto_cardinal.Punto_cardinal) #reseteo de c  
Orden2 = V
for i in d:   
  Orden2 = Orden2.replace(to_replace = c[i], value = d[i])
#Segmentar datos de viento horario con direccion en puntos cardinales
df2 = pd.DataFrame (df.loc[:,['Fecha','Velocidad_viento']])
df2 = df2.assign(Tipo_cardinal=V) 
df2 = df2.assign(Orden2 = Orden2)
df2['Dia'] = pd.to_datetime(df.Fecha).dt.date
df2['Hora'] = pd.to_datetime(df.Fecha).dt.time

Dias de lluvia y estacion meteorologica

In [60]:
# Preparando datos
Consulta1['Lluvia'] = np.where(Consulta1.Pd == 'nan', np.nan, np.where(Consulta1.Pd == 0, 'no','si')) 
Estacion_astronomica['Fecha'] = Estacion_astronomica['Fecha'].round('d')
Consulta1['Estacion'] = np.where((Consulta1['Fecha'] >= Estacion_astronomica.loc[0,'Fecha']) & (Consulta1['Fecha'] < Estacion_astronomica.loc[1,'Fecha'])|(Consulta1['Fecha'] >= (Estacion_astronomica.loc[4,'Fecha'])),'Summer',np.where((Consulta1['Fecha'] >= Estacion_astronomica.loc[1,'Fecha']) & (Consulta1['Fecha'] < Estacion_astronomica.loc[2,'Fecha']),'Autumn', np.where((Consulta1['Fecha'] >= Estacion_astronomica.loc[2,'Fecha']) & (Consulta1['Fecha'] < Estacion_astronomica.loc[3,'Fecha']),'Winter','Spring')))

Fecha
2021-01-31    24
2021-02-28    10
2021-03-31    21
2021-04-30    10
2021-05-31     1
2021-06-30     0
2021-07-31     0
2021-08-31     0
2021-09-30     1
2021-10-31     5
2021-11-30     4
2021-12-31     6
Freq: M, Name: Lluvia, dtype: int64

Exportar datos

In [61]:
# Exportando datos
df2.to_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\df2.csv')
df[['Dia','Hora','Th','TC_Th']].to_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\dfTh.csv')
df[['Dia','Hora','Hh','TC_Hh']].to_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\dfHh.csv')
df[['Dia','Hora','Ph','TC_Ph']].to_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\dfPh.csv')
Consulta1[['Fecha','Lluvia','Estacion']].to_csv(r'C:\Users\jr-sa\Desktop\EstacionMeteo\dfLluviaEstacion.csv') 